<a href="https://colab.research.google.com/github/chasecaker/airport/blob/main/airport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ТАБЛИЦЫ

In [ ]:
from peewee import *
from datetime import *
import sqlite3
from playhouse.db_url import connect  # type: ignore
from playhouse.shortcuts import model_to_dict  # type: ignore
from playhouse.reflection import generate_models  # type: ignore
from pydantic import BaseModel as PydanticBaseModel
from typing import List, Optional
from tabulate import tabulate

db_filename = 'airport.db'
source_db = SqliteDatabase(db_filename)
source_models = generate_models(source_db)

orm_db = SqliteDatabase('airport_orm.db')

class BaseModel(Model):
    class Meta:
        database = orm_db

class airline(BaseModel):
    id = AutoField()
    name = CharField(unique=True)
    country = CharField(null=True)
    contact_info = CharField(null=True)


class terminal(BaseModel):
    id = AutoField()
    name = CharField(unique=True)
    floor_count = IntegerField(null=True)


class shift(BaseModel):
    id = IntegerField(primary_key=True)
    start_time = CharField()
    end_time = CharField()
    shift_type = CharField(null=True)


class runway(BaseModel):
    id = AutoField()
    code = CharField(unique=True)
    length_meters = IntegerField(null=True)
    is_active = BooleanField(default=True)


class gate(BaseModel):
    id = AutoField()
    gate_code = CharField(unique=True)
    terminal = ForeignKeyField(terminal, backref='gates')


class flight(BaseModel):
    id = AutoField()
    flight_number = CharField(unique=True)
    departure_time = CharField()
    arrival_time = CharField()
    from_city = CharField(null=True)
    to_city = CharField(null=True)
    airline = ForeignKeyField(airline, backref='flights')
    gate = ForeignKeyField(gate, backref='flights', null=True)
    runway = ForeignKeyField(runway, backref='flights', null=True)


class passenger(BaseModel):
    id = AutoField()
    full_name = CharField()
    passport_number = CharField(unique=True)
    birth_date = CharField()
    nationality = CharField(null=True)
    phone = CharField(null=True)
    email = CharField(null=True)


class seat(BaseModel):
    id = AutoField()
    flight = ForeignKeyField(flight, backref='seats')
    seat_number = CharField()
    class_type = CharField()
    is_taken = BooleanField(null=True)


class ticket(BaseModel):
    id = AutoField()
    passenger = ForeignKeyField(passenger, backref='tickets')
    flight = ForeignKeyField(flight, backref='tickets')
    seat = ForeignKeyField(seat, backref='tickets', null=True)
    price = FloatField(constraints=[Check('price > 0')])
    purchase_date = CharField()


class baggageBelt(BaseModel):
    id = AutoField()
    belt_number = CharField(unique=True)
    terminal = ForeignKeyField(terminal, backref='baggage_belts', null=True)


class baggage(BaseModel):
    id = AutoField()
    passenger = ForeignKeyField(passenger, backref='baggages')
    flight = ForeignKeyField(flight, backref='baggages')
    weight = FloatField(null=True)
    status = CharField(null=True)
    baggage_belt = ForeignKeyField(baggageBelt, backref='baggages', null=True)

class cargo(BaseModel):
    id = AutoField()
    flight = ForeignKeyField(flight, backref='cargos')
    description = TextField(null=True)
    weight = FloatField(null=True)
    owner_company = CharField(null=True)
    status = CharField(null=True)

class employee(BaseModel):
    id = AutoField()
    full_name = CharField()
    position = CharField()
    gate = ForeignKeyField(gate, backref='employees', null=True)
    terminal = ForeignKeyField(terminal, backref='employees', null=True)
    shift = ForeignKeyField(shift, backref='employees', null=True)
    contact_number = CharField(null=True)


class equipment(BaseModel):
    id = AutoField()
    name = CharField(null=True)
    type = CharField(null=True)
    status = CharField(null=True)
    employee = ForeignKeyField(employee, backref='equipments', null=True)


class equipmentMaintenance(BaseModel):
    id = AutoField()
    equipment = ForeignKeyField(equipment, backref='maintenances')
    start_time = CharField(null=True)
    end_time = CharField(null=True)
    description = TextField(null=True)


class cleaningSchedule(BaseModel):
    id = AutoField()
    area_type = CharField(null=True)
    scheduled_at = CharField(null=True)
    employee = ForeignKeyField(employee, backref='cleaning_schedules')
    status = CharField(null=True)

class flightDelay(BaseModel):
    id = AutoField()
    flight = ForeignKeyField(flight, backref='delays')
    reason = TextField(null=True)
    duration_min = CharField(null=True)


class flightHistory(BaseModel):
    id = AutoField()
    flight = ForeignKeyField(flight, backref='history')
    status = CharField()
    updated_at = CharField(default=datetime.now)


class airplane(BaseModel):
    id = AutoField()
    flight = ForeignKeyField(flight, backref='airplanes', null=True)
    airline = ForeignKeyField(airline, backref='airplanes', null=True)
    runway = ForeignKeyField(runway, backref='airplanes', null=True)


class service(BaseModel):
    id = AutoField()
    name = CharField()
    description = TextField(null=True)
    location = CharField(null=True)
    price = FloatField(null=True)


class foodOrder(BaseModel):
    id = AutoField()
    ticket = ForeignKeyField(ticket, backref='food_orders')
    meal_type = CharField(null=True)
    ordered_at = CharField(null=True)


class payment(BaseModel):
    id = AutoField()
    ticket = ForeignKeyField(ticket, backref='payments')
    method = CharField(null=True)
    status = CharField(null=True)


class healthCheck(BaseModel):
    id = AutoField()
    passenger = ForeignKeyField(passenger, backref='health_checks')
    status = CharField(null=True)


class loungeAccess(BaseModel):
    id = AutoField()
    passenger = ForeignKeyField(passenger, backref='lounge_accesses')
    access = BooleanField(null=True)


class passengerService(BaseModel):
    id = AutoField()
    passenger = ForeignKeyField(passenger, backref='passenger_services')
    service = ForeignKeyField(service, backref='passenger_services')
    time_used = CharField(null=True)


def migrate_table(source_model, target_model, batch_size=100):
    try:
        total = source_model.select().count()
        if total == 0:
            print(f"Таблица {source_model._meta.table_name} пуста, пропускаем")
            return

        print(f"Перенос {source_model._meta.table_name}: {total} записей...")
        success = 0
        errors = 0

        for i in range(0, total, batch_size):
            with orm_db.atomic():
                for item in source_model.select().offset(i).limit(batch_size):
                    try:
                        data = item.__data__

                        id_field = target_model._meta.primary_key.name
                        if id_field in data:
                            del data[id_field]

                        unique_fields = [f.name for f in target_model._meta.fields.values()
                                       if f.unique and not isinstance(f, AutoField)]

                        for field_name in unique_fields:
                            if field_name in data and data[field_name] is None:
                                data[field_name] = f"TEMP_NULL_{item.id}" if hasattr(item, 'id') else f"TEMP_NULL_{i}"

                        for field_name in list(data.keys()):
                            if data[field_name] is None:
                                field = target_model._meta.fields.get(field_name)
                                if field and not field.null and not isinstance(field, AutoField):
                                    if isinstance(field, CharField):
                                        data[field_name] = ""
                                    elif isinstance(field, IntegerField):
                                        data[field_name] = 0
                                    elif isinstance(field, FloatField):
                                        data[field_name] = 0.0
                                    elif isinstance(field, BooleanField):
                                        data[field_name] = False
                                    elif isinstance(field, ForeignKeyField):
                                        del data[field_name]

                        target_model.create(**data)
                        success += 1
                    except Exception as e:
                        errors += 1
                        print(f"Ошибка при переносе записи {getattr(item, 'id', 'None')}: {str(e)}")
                        continue
            print(f"Прогресс: {min(i + batch_size, total)}/{total} | Успешно: {success} | Ошибок: {errors}")
    except Exception as e:
        print(f"Критическая ошибка при переносе таблицы {source_model._meta.table_name}: {str(e)}")


with orm_db:
    orm_db.drop_tables(BaseModel.__subclasses__())
    orm_db.create_tables(BaseModel.__subclasses__())

tables_order = [
    ('airline', airline),
    ('terminal', terminal),
    ('runway', runway),
    ('gate', gate),
    ('airplane', airplane),
    ('flight', flight),
    ('flightHistory', flightHistory),
    ('flightDelay', flightDelay),
    ('passenger', passenger),
    ('seat', seat),
    ('ticket', ticket),
    ('baggageBelt', baggageBelt),
    ('baggage', baggage),
    ('cargo', cargo),
    ('employee', employee),
    ('shift', shift),
    ('equipment', equipment),
    ('equipmentMaintenance', equipmentMaintenance),
    ('cleaningSchedule', cleaningSchedule),
    ('service', service),
    ('foodOrder', foodOrder),
    ('payment', payment),
    ('healthCheck', healthCheck),
    ('loungeAccess', loungeAccess),
    ('passengerService', passengerService)
]


for table_name, model in tables_order:
    if table_name in source_models:
        migrate_table(source_models[table_name], model)
    else:
        print(f"Таблица {table_name} не найдена в исходной БД")

source_db.close()
orm_db.close()

print("\nПеренос данных завершен!")

orm_db = SqliteDatabase('airport_orm.db')
orm_db.connect()

tables = orm_db.get_tables()

for table_name in tables:
    print(f"\n=== {table_name.upper()} ===")
    try:
        cursor = orm_db.execute_sql(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]
        print(tabulate(rows, headers=headers, tablefmt='grid'))
    except Exception as e:
        print(f"Ошибка при выводе таблицы {table_name}: {e}")

Перенос airline: 10 записей...
Прогресс: 10/10 | Успешно: 10 | Ошибок: 0
Перенос terminal: 10 записей...
Прогресс: 10/10 | Успешно: 10 | Ошибок: 0
Перенос runway: 10 записей...
Прогресс: 10/10 | Успешно: 10 | Ошибок: 0
Перенос gate: 12 записей...
Прогресс: 12/12 | Успешно: 12 | Ошибок: 0
Перенос airplane: 5 записей...
Прогресс: 5/5 | Успешно: 5 | Ошибок: 0
Перенос flight: 10 записей...
Прогресс: 10/10 | Успешно: 10 | Ошибок: 0
Перенос flightHistory: 10 записей...
Прогресс: 10/10 | Успешно: 10 | Ошибок: 0
Перенос flightDelay: 3 записей...
Прогресс: 3/3 | Успешно: 3 | Ошибок: 0
Перенос passenger: 10 записей...
Прогресс: 10/10 | Успешно: 10 | Ошибок: 0
Перенос seat: 10 записей...
Прогресс: 10/10 | Успешно: 10 | Ошибок: 0
Перенос ticket: 10 записей...
Прогресс: 10/10 | Успешно: 10 | Ошибок: 0
Перенос baggageBelt: 13 записей...
Прогресс: 13/13 | Успешно: 13 | Ошибок: 0
Перенос baggage: 13 записей...
Прогресс: 13/13 | Успешно: 13 | Ошибок: 0
Перенос cargo: 10 записей...
Прогресс: 10/10 | Усп

In [ ]:
!pip install fastapi uvicorn python-multipart nest_asyncio pyngrok

РАЗНИЦА СР СТОИМ БИЛЕТА И СЕРВИСА ПАССАЖИРА

In [ ]:
ticket_price = (ticket
                .select(
                    ticket.passenger_id,
                    fn.AVG(ticket.price).alias('avg_ticket_price')
                )
                .group_by(ticket.passenger_id))

service_price = (service
                 .select(
                     service.id,
                     fn.AVG(service.price).alias('avg_service_price')
                 )
                 .group_by(service.id))

query = (passengerService
         .select(
             passenger.full_name.alias('passenger_name'),
             service.name.alias('service_name'),
             ticket_price.c.avg_ticket_price,
             service_price.c.avg_service_price,
             (ticket_price.c.avg_ticket_price - service_price.c.avg_service_price).alias('price_difference')
         )
         .join(passenger, on=(passengerService.passenger_id == passenger.id))
         .join(service, on=(passengerService.service_id == service.id))
         .join(ticket_price, on=(passengerService.passenger_id == ticket_price.c.passenger_id))
         .join(service_price, on=(passengerService.service_id == service_price.c.id))
         .order_by((ticket_price.c.avg_ticket_price - service_price.c.avg_service_price).desc()))

import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,passenger_name,service_name,avg_ticket_price,avg_service_price,price_difference
0,Петрова Анна Сергеевна,Быстрая регистрация,15700.0,2500.0,13200.0
1,Мякушкин Евгений Олегович,Камера хранения,15600.0,4000.0,11600.0
2,Петрова Анна Сергеевна,Бизнес-зал,15700.0,5000.0,10700.0
3,Сидоров Дмитрий Николаевич,Перевозка багажа,9800.0,1500.0,8300.0
4,Раднаева Елизавета Сергеевна,Премиум-паркинг,3800.0,3000.0,800.0


In [ ]:
!pip install flask

ЗАГРУЖЕННОСТЬ БАГАЖНЫХ ЛЕНТ

In [ ]:
query = (baggageBelt
         .select(
             terminal.name.alias('terminal'),
             baggageBelt.belt_number,
             fn.COUNT(baggage.id).alias('baggage_count'),
             fn.AVG(baggage.weight).alias('avg_weight'),
             fn.MAX(baggage.weight).alias('max_weight'),
             airline.name.alias('airlines')
         )
         .join(terminal, on=(baggageBelt.terminal_id == terminal.id))
         .switch(baggageBelt)
         .join(baggage, join_type=JOIN.LEFT_OUTER, on=(baggageBelt.id == baggage.baggage_belt_id))
         .join(flight, join_type=JOIN.LEFT_OUTER, on=(baggage.flight_id == flight.id))
         .join(airline, join_type=JOIN.LEFT_OUTER, on=(flight.airline_id == airline.id))
         .group_by(baggageBelt.id, terminal.name, baggageBelt.belt_number)
         .order_by(fn.MAX(baggage.weight).desc())
         .limit(8))

import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,terminal,belt_number,baggage_count,avg_weight,max_weight,airlines
0,Южный терминал,S2,2,24.55,33.4,Аэрофлот
1,Южный терминал,S1,1,32.00,32.0,Победа
2,Грузовой терминал,G1,1,30.50,30.5,Уральские авиалинии
3,Центральный терминал,C1,2,21.75,28.9,Победа
4,Терминал B,B1,1,24.80,24.8,S7 Airlines
5,Северный терминал,N1,1,23.50,23.5,S7 Airlines
6,Терминал А,A1,1,22.10,22.1,Россия
7,Центральный терминал,C2,2,14.80,19.5,Победа


ВСЕ ПЕРЕСЕКАЮЩИЕСЯ ОБСЛУЖИВАНИЯ ОБОРУДОВАНИЯ

In [ ]:
em1 = equipmentMaintenance.alias('em1')
em2 = equipmentMaintenance.alias('em2')

overlapping_count = fn.COUNT(em2.id) - 1

query = (em1
         .select(
             em1.id.alias('maintenance_id'),
             em1.equipment_id,
             em1.start_time,
             em1.end_time,
             overlapping_count.alias('overlapping_count')
         )
         .join(em2, JOIN.INNER, on=(
             (em1.start_time < em2.end_time) &
             (em1.end_time > em2.start_time)
         ))
         .group_by(em1.id, em1.equipment_id, em1.start_time, em1.end_time)
         .having(overlapping_count > 0)
         .order_by(
             overlapping_count.desc(),
             em1.start_time
         ))

import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,maintenance_id,equipment,start_time,end_time,overlapping_count
0,1,3,2023-12-15 09:00:00,2023-12-15 11:30:00,1
1,2,5,2023-12-15 10:00:00,2023-12-15 14:00:00,1
2,3,8,2023-12-16 08:00:00,2023-12-16 15:45:00,1
3,4,4,2023-12-16 13:00:00,2023-12-16 15:45:00,1


ТОП 5 ДЛИТЕЛЬНОСТЬ ПОЧИНКИ ОБОРУДОВАНИЯ

In [ ]:
latest_maintenance = (equipmentMaintenance
                    .select(
                        equipmentMaintenance.equipment_id,
                        fn.MAX(equipmentMaintenance.start_time).alias('max_start_time'))
                    .group_by(equipmentMaintenance.equipment_id)
                    .alias('latest_maint'))

query = (equipmentMaintenance
         .select(
             equipment.name,
             equipment.type,
             equipmentMaintenance.start_time,
             equipmentMaintenance.end_time,
             fn.ROUND(
                 (fn.Julianday(equipmentMaintenance.end_time) -
                  fn.Julianday(equipmentMaintenance.start_time)) * 24 * 60,
                 1).alias('duration_minutes')
         )
         .join(latest_maintenance, on=(
             (equipmentMaintenance.equipment_id == latest_maintenance.c.equipment_id) &
             (equipmentMaintenance.start_time == latest_maintenance.c.max_start_time)
         ))
         .join(equipment, on=(equipmentMaintenance.equipment_id == equipment.id))
         .where(equipmentMaintenance.equipment_id.is_null(False))
         .order_by(SQL('duration_minutes').desc())
         .limit(5))

import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,name,type,start_time,end_time,duration_minutes
0,Гидравлический подъемник,Техника,2023-12-15 10:00:00,2023-12-15 14:00:00,240.0
1,Сканер штрих-кодов,IT,2023-12-19 14:00:00,2023-12-19 17:30:00,210.0
2,Принтер посадочных талонов,IT,2023-12-16 13:00:00,2023-12-16 15:45:00,165.0
3,Сканер багажа,Безопасность,2023-12-15 09:00:00,2023-12-15 11:30:00,150.0
4,Металлодетектор,Безопасность,2023-12-17 14:00:00,2023-12-17 16:30:00,150.0


ЗДОРОВЫЕ ПАССАЖИРЫ ИМЕЮЩИЕ ДОСТУП В ЛАУНДЖЗОНУ  

In [ ]:
has_lounge_access = Case(None, [
    (loungeAccess.access == 1, Value('yes'))
], Value('no'))

query = (passenger
         .select(
             passenger.full_name,
             passenger.phone,
             healthCheck.status.alias('health_status'),
             has_lounge_access.alias('has_lounge_access')
         )
         .join(healthCheck, on=(passenger.id == healthCheck.passenger_id))
         .join(loungeAccess, on=(passenger.id == loungeAccess.passenger_id))
         .where(
             (loungeAccess.access == 1) &
             (healthCheck.status == 'passed')
         )
         .order_by(passenger.full_name))

import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,full_name,phone,health_status,has_lounge_access
0,Kim Chang Yin,+82565345656,passed,yes
1,Lee San Dan,+44878726543,passed,yes
2,Иванов Иван Иванович,+79161234567,passed,yes
3,Петров Петр Петрович,+79837827827,passed,yes
4,Пехота Игорь Степанович,+79887773627,passed,yes
5,Раднаева Елизавета Сергеевна,+79817267622,passed,yes


БЛЮДО КОТОРОЕ БОЛЬШЕ ВСЕГО ПОДАВАЛОСЬ НЕПРЕРЫВНО

In [ ]:
query = (foodOrder
         .select(
             foodOrder.meal_type,
             fn.COUNT(foodOrder.id).alias('max_streak'))
         .group_by(foodOrder.meal_type)
         .order_by(fn.COUNT(foodOrder.id).desc())
         .limit(1))


import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,meal_type,max_streak
0,Мясо,4


РАНЖИРОВАНИЕ ПО ТРАФИКУ ПАССАЖИРОВ

In [ ]:
query = (terminal
          .select(
              terminal.name.alias('terminal_name'),
              fn.COUNT(fn.DISTINCT(passenger.id)).alias('passengers_count'),
              (fn.COUNT(fn.DISTINCT(flight.id)) /
               fn.COUNT(fn.DISTINCT(gate.id))).alias('flights_per_gate'),
              fn.DENSE_RANK().over(order_by=[fn.COUNT(fn.DISTINCT(passenger.id)).desc()]).alias('passenger_traffic_rank'))
          .join(gate, JOIN.LEFT_OUTER, on=(terminal.id == gate.terminal_id))
          .switch(terminal)
          .join(flight, JOIN.LEFT_OUTER, on=(gate.id == flight.gate_id))
          .switch(flight)
          .join(ticket, JOIN.LEFT_OUTER, on=(flight.id == ticket.flight_id))
          .switch(ticket)
          .join(passenger, JOIN.LEFT_OUTER, on=(ticket.passenger_id == passenger.id))
          .group_by(terminal.id, terminal.name)
          .order_by(SQL('passenger_traffic_rank'))
          .dicts())

import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,terminal_name,passengers_count,flights_per_gate,passenger_traffic_rank
0,Южный терминал,3,1.0,1
1,Северный терминал,2,1.0,2
2,Центральный терминал,1,1.0,3
3,Терминал А,1,1.0,3
4,Терминал B,1,1.0,3
5,VIP терминал,1,1.0,3
6,Грузовой терминал,1,1.0,3
7,Международный терминал,0,NaN,4
8,Терминал D,0,NaN,4
9,Восточный терминал,0,NaN,4


РАНЖИРОВАНИЕ ВНУТРИ КОМПАНИИ ПО ГРУЗУ

In [ ]:
company_totals = (cargo
                 .select(
                     cargo.owner_company,
                     fn.round(fn.SUM(cargo.weight), 2).alias('company_weight'))
                 .group_by(cargo.owner_company)
                 .alias('ct'))

query = (cargo
          .select(
              cargo.owner_company,
              cargo.description.alias('heaviest_cargo_type'),
              fn.round(fn.SUM(cargo.weight), 2).alias('type_weight'),
              company_totals.c.company_weight,
              fn.RANK().over(
                  partition_by=[cargo.owner_company],
                  order_by=[fn.SUM(cargo.weight).desc()]).alias('rnk'),
              fn.round(100.0 * fn.SUM(cargo.weight) /
                      company_totals.c.company_weight, 2).alias('weight_share_percent'))
          .join(company_totals, on=(cargo.owner_company == company_totals.c.owner_company))
          .group_by(cargo.owner_company, cargo.description, company_totals.c.company_weight)
          .order_by(cargo.owner_company.desc(), fn.SUM(cargo.weight).desc())
          .dicts())


import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,owner_company,heaviest_cargo_type,type_weight,company_weight,rnk,weight_share_percent
0,Черкизово,Продукты питания,150.8,150.8,1,100.00
1,Фармасинтез,Медицинское оборудование,320.0,320.0,1,100.00
2,М.Видео,Электронные компоненты,450.5,670.6,1,67.18
3,М.Видео,Бытовая техника,220.1,670.6,2,32.82
4,Кнауф,Строительные материалы,410.3,410.3,1,100.00
5,БТК Групп,Одежда и текстиль,365.8,365.8,1,100.00
6,АвтоВАЗ,Инструменты,380.9,787.0,1,48.40
7,АвтоВАЗ,Оборудование,280.7,787.0,2,35.67
8,АвтоВАЗ,Автозапчасти,125.4,787.0,3,15.93


САМЫЕ ДЛИННЫЕ ИНТЕРВАЛЫ МЕЖДУ УБОРКАМИ ПО ЗОНАМ

In [ ]:
query = (cleaningSchedule
         .select(
             cleaningSchedule.area_type,
             cleaningSchedule.scheduled_at,
             fn.LAG(cleaningSchedule.scheduled_at).over(
                 partition_by=[cleaningSchedule.area_type],
                 order_by=[cleaningSchedule.scheduled_at]).alias('previous_time'),
             fn.round(
                 (fn.JULIANDAY(cleaningSchedule.scheduled_at) -
                  fn.JULIANDAY(fn.LAG(cleaningSchedule.scheduled_at).over(
                      partition_by=[cleaningSchedule.area_type],
                      order_by=[cleaningSchedule.scheduled_at]))) * 24 * 60, 2
             ).alias('minutes_since_last'))
         .where(cleaningSchedule.status == 'planned')
         .order_by(SQL('minutes_since_last DESC')))


import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,area_type,scheduled_at,previous_time,minutes_since_last
0,terminal,2023-12-15 13:00:00,2023-12-15 08:00:00,300.0
1,terminal,2023-12-15 18:00:00,2023-12-15 13:00:00,300.0
2,terminal,2023-12-15 23:00:00,2023-12-15 18:00:00,300.0
3,terminal,2023-12-15 08:00:00,None,NaN


РЕЙСЫ СО СМЕНОЙ СТАТУСОВ ЗА ДЕНЬ

In [ ]:
status_changes = (flightHistory
                 .select(
                     flightHistory.flight_id,
                     fn.date(flightHistory.updated_at).alias('change_day'),
                     fn.count(flightHistory.id).alias('change_count'))
                 .group_by(flightHistory.flight_id, SQL('change_day'))
                 .alias('sc'))

max_changes = (status_changes
              .select(
                  status_changes.c.flight_id,
                  fn.max(status_changes.c.change_count).alias('max_changes'))
              .group_by(status_changes.c.flight_id)
              .alias('mcf'))

last_status = (flightHistory
              .select(
                  flightHistory.flight_id,
                  flightHistory.status,
                  flightHistory.updated_at,
                  fn.row_number().over(
                      partition_by=[flightHistory.flight_id],
                      order_by=[flightHistory.updated_at.desc()]).alias('rn'))
              .where(SQL('rn = 1'))
              .alias('ls'))

query = (flightHistory
        .select(
            flightHistory.flight_id,
            fn.max(fn.count()).over(partition_by=[
                flightHistory.flight_id,
                fn.date(flightHistory.updated_at)]).alias('max_changes'),
            flightHistory.status.alias('last_status'),
            flightHistory.updated_at.alias('last_update_time'))
        .group_by(flightHistory.flight_id)
        .order_by(SQL('max_changes DESC'), SQL('last_update_time DESC')))


import pandas as pd
df = pd.DataFrame(list(query.dicts().execute()))
df

,flight,max_changes,last_status,last_update_time
0,3,3,arrived,2023-12-15 11:10:00
1,7,2,boarding,2023-12-15 07:30:00
2,8,1,departed,2023-12-15 10:50:00
3,10,1,delayed,2023-12-15 10:15:00
4,1,1,departed,2023-12-15 08:05:00
5,2,1,scheduled,2023-12-14 10:05:00
6,5,1,scheduled,2023-12-14 10:00:00


апишки

In [ ]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import uvicorn
from pyngrok import ngrok, conf
import threading
import time
import nest_asyncio

nest_asyncio.apply()
app = FastAPI()

ngrok.set_auth_token("2xwhpx8xQnOZm3Awugcp02fq2jW_PAVYTKBJRJeLeDsn8LpR")

db = SqliteDatabase('airport.db')

# 1.
@app.get("/max_streak_meal")
async def get_max_streak_meal():
    try:
        query = (foodOrder
         .select(
             foodOrder.meal_type,
             fn.COUNT(foodOrder.id).alias('max_streak'))
         .group_by(foodOrder.meal_type)
         .order_by(fn.COUNT(foodOrder.id).desc())
         .limit(1))

        df = pd.DataFrame(list(query.dicts()))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

# 2.
@app.get("/cargo_ranking")
async def get_cargo_ranking():
    try:
        company_totals = (cargo
                 .select(
                     cargo.owner_company,
                     fn.round(fn.SUM(cargo.weight), 2).alias('company_weight'))
                 .group_by(cargo.owner_company)
                 .alias('ct'))

        query = (cargo
          .select(
              cargo.owner_company,
              cargo.description.alias('heaviest_cargo_type'),
              fn.round(fn.SUM(cargo.weight), 2).alias('type_weight'),
              company_totals.c.company_weight,
              fn.RANK().over(
                  partition_by=[cargo.owner_company],
                  order_by=[fn.SUM(cargo.weight).desc()]).alias('rnk'),
              fn.round(100.0 * fn.SUM(cargo.weight) /
                      company_totals.c.company_weight, 2).alias('weight_share_percent'))
          .join(company_totals, on=(cargo.owner_company == company_totals.c.owner_company))
          .group_by(cargo.owner_company, cargo.description, company_totals.c.company_weight)
          .order_by(cargo.owner_company.desc(), fn.SUM(cargo.weight).desc())
          .dicts())

        df = pd.DataFrame(list(query))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

# 3.
@app.get("/status_analysis")
async def get_status_analysis():
    try:
        status_changes = (flightHistory
                 .select(
                     flightHistory.flight_id,
                     fn.date(flightHistory.updated_at).alias('change_day'),
                     fn.count(flightHistory.id).alias('change_count'))
                 .group_by(flightHistory.flight_id, SQL('change_day'))
                 .alias('sc'))

        max_changes = (status_changes
              .select(
                  status_changes.c.flight_id,
                  fn.max(status_changes.c.change_count).alias('max_changes'))
              .group_by(status_changes.c.flight_id)
              .alias('mcf'))

        last_status = (flightHistory
              .select(
                  flightHistory.flight_id,
                  flightHistory.status,
                  flightHistory.updated_at,
                  fn.row_number().over(
                      partition_by=[flightHistory.flight_id],
                      order_by=[flightHistory.updated_at.desc()]).alias('rn'))
              .where(SQL('rn = 1'))
              .alias('ls'))

        query = (flightHistory
        .select(
            flightHistory.flight_id,
            fn.max(fn.count()).over(partition_by=[
                flightHistory.flight_id,
                fn.date(flightHistory.updated_at)]).alias('max_changes'),
            flightHistory.status.alias('last_status'),
            flightHistory.updated_at.alias('last_update_time'))
        .group_by(flightHistory.flight_id)
        .order_by(SQL('max_changes DESC'), SQL('last_update_time DESC')))

        df = pd.DataFrame(list(query))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

# 4.
@app.get("/intervals")
async def get_intervals():
    try:
        query = (cleaningSchedule
         .select(
             cleaningSchedule.area_type,
             cleaningSchedule.scheduled_at,
             fn.LAG(cleaningSchedule.scheduled_at).over(
                 partition_by=[cleaningSchedule.area_type],
                 order_by=[cleaningSchedule.scheduled_at]).alias('previous_time'),
             fn.round(
                 (fn.JULIANDAY(cleaningSchedule.scheduled_at) -
                  fn.JULIANDAY(fn.LAG(cleaningSchedule.scheduled_at).over(
                      partition_by=[cleaningSchedule.area_type],
                      order_by=[cleaningSchedule.scheduled_at]))) * 24 * 60, 2
             ).alias('minutes_since_last'))
         .where(cleaningSchedule.status == 'planned')
         .order_by(SQL('minutes_since_last DESC')))

        df = pd.DataFrame(list(query.dicts()))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

# 5.
@app.get("/healthy_passenger")
async def get_healthy_passenger():
    try:
        has_lounge_access = Case(None, [
    (loungeAccess.access == 1, Value('yes'))
], Value('no'))

        query = (passenger
         .select(
             passenger.full_name,
             passenger.phone,
             healthCheck.status.alias('health_status'),
             has_lounge_access.alias('has_lounge_access')
         )
         .join(healthCheck, on=(passenger.id == healthCheck.passenger_id))
         .join(loungeAccess, on=(passenger.id == loungeAccess.passenger_id))
         .where(
             (loungeAccess.access == 1) &
             (healthCheck.status == 'passed')
         )
         .order_by(passenger.full_name))

        df = pd.DataFrame(list(query.dicts()))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}
# 6.
@app.get("/traffic_ranking")
async def get_traffic_ranking():
    try:
        query = (terminal
          .select(
              terminal.name.alias('terminal_name'),
              fn.COUNT(fn.DISTINCT(passenger.id)).alias('passengers_count'),
              (fn.COUNT(fn.DISTINCT(flight.id)) /
               fn.COUNT(fn.DISTINCT(gate.id))).alias('flights_per_gate'),
              fn.DENSE_RANK().over(order_by=[fn.COUNT(fn.DISTINCT(passenger.id)).desc()]).alias('passenger_traffic_rank'))
          .join(gate, JOIN.LEFT_OUTER, on=(terminal.id == gate.terminal_id))
          .switch(terminal)
          .join(flight, JOIN.LEFT_OUTER, on=(gate.id == flight.gate_id))
          .switch(flight)
          .join(ticket, JOIN.LEFT_OUTER, on=(flight.id == ticket.flight_id))
          .switch(ticket)
          .join(passenger, JOIN.LEFT_OUTER, on=(ticket.passenger_id == passenger.id))
          .group_by(terminal.id, terminal.name)
          .order_by(SQL('passenger_traffic_rank'))
          .dicts())

        df = pd.DataFrame(list(query.dicts()))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

# 7.
@app.get("/top_5_longest_maintence")
async def get_top_5_longest_maintence():
    try:
        latest_maintenance = (equipmentMaintenance
                    .select(
                        equipmentMaintenance.equipment_id,
                        fn.MAX(equipmentMaintenance.start_time).alias('max_start_time'))
                    .group_by(equipmentMaintenance.equipment_id)
                    .alias('latest_maint'))

        query = (equipmentMaintenance
         .select(
             equipment.name,
             equipment.type,
             equipmentMaintenance.start_time,
             equipmentMaintenance.end_time,
             fn.ROUND(
                 (fn.Julianday(equipmentMaintenance.end_time) -
                  fn.Julianday(equipmentMaintenance.start_time)) * 24 * 60,
                 1).alias('duration_minutes')
         )
         .join(latest_maintenance, on=(
             (equipmentMaintenance.equipment_id == latest_maintenance.c.equipment_id) &
             (equipmentMaintenance.start_time == latest_maintenance.c.max_start_time)
         ))
         .join(equipment, on=(equipmentMaintenance.equipment_id == equipment.id))
         .where(equipmentMaintenance.equipment_id.is_null(False))
         .order_by(SQL('duration_minutes').desc())
         .limit(5))


        df = pd.DataFrame(list(query.dicts()))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

# 8.
@app.get("/baggage_belt_capacity")
async def get_baggage_belt_capacity():
    try:
        query = (baggageBelt
         .select(
             terminal.name.alias('terminal'),
             baggageBelt.belt_number,
             fn.COUNT(baggage.id).alias('baggage_count'),
             fn.AVG(baggage.weight).alias('avg_weight'),
             fn.MAX(baggage.weight).alias('max_weight'),
             airline.name.alias('airlines')
         )
         .join(terminal, on=(baggageBelt.terminal_id == terminal.id))
         .switch(baggageBelt)
         .join(baggage, join_type=JOIN.LEFT_OUTER, on=(baggageBelt.id == baggage.baggage_belt_id))
         .join(flight, join_type=JOIN.LEFT_OUTER, on=(baggage.flight_id == flight.id))
         .join(airline, join_type=JOIN.LEFT_OUTER, on=(flight.airline_id == airline.id))
         .group_by(baggageBelt.id, terminal.name, baggageBelt.belt_number)
         .order_by(fn.MAX(baggage.weight).desc()))

        df = pd.DataFrame(list(query.dicts()))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

# 9.
@app.get("/maintence_intersections")
async def get_maintence_intersections():
    try:
        em1 = equipmentMaintenance.alias('em1')
        em2 = equipmentMaintenance.alias('em2')

        overlapping_count = fn.COUNT(em2.id) - 1

        query = (em1
         .select(
             em1.id.alias('maintenance_id'),
             em1.equipment_id,
             em1.start_time,
             em1.end_time,
             overlapping_count.alias('overlapping_count')
         )
         .join(em2, JOIN.INNER, on=(
             (em1.start_time < em2.end_time) &
             (em1.end_time > em2.start_time)
         ))
         .group_by(em1.id, em1.equipment_id, em1.start_time, em1.end_time)
         .having(overlapping_count > 0)
         .order_by(
             overlapping_count.desc(),
             em1.start_time
         ))


        df = pd.DataFrame(list(query.dicts()))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

# 10.
@app.get("/price_difference")
async def get_price_difference():
    try:
        ticket_price = (ticket
                .select(
                    ticket.passenger_id,
                    fn.AVG(ticket.price).alias('avg_ticket_price')
                )
                .group_by(ticket.passenger_id))

        service_price = (service
                 .select(
                     service.id,
                     fn.AVG(service.price).alias('avg_service_price')
                 )
                 .group_by(service.id))

        query = (passengerService
         .select(
             passenger.full_name.alias('passenger_name'),
             service.name.alias('service_name'),
             ticket_price.c.avg_ticket_price,
             service_price.c.avg_service_price,
             (ticket_price.c.avg_ticket_price - service_price.c.avg_service_price).alias('price_difference')
         )
         .join(passenger, on=(passengerService.passenger_id == passenger.id))
         .join(service, on=(passengerService.service_id == service.id))
         .join(ticket_price, on=(passengerService.passenger_id == ticket_price.c.passenger_id))
         .join(service_price, on=(passengerService.service_id == service_price.c.id))
         .order_by((ticket_price.c.avg_ticket_price - service_price.c.avg_service_price).desc()))

        df = pd.DataFrame(list(query.dicts()))
        return JSONResponse(df.to_dict(orient="records"))
    except Exception as e:
        return {"error": str(e), "type": type(e).__name__}

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8001)

if __name__ == "__main__":
    ngrok.kill()

    thread = threading.Thread(target=run_server, daemon=True)
    thread.start()
    time.sleep(2)

    try:
        public_url = ngrok.connect(8001)
        print("Готово! Доступные эндпоинты:")
        print(f"1. Анализ питания: {public_url.public_url}/max_streak_meal")
        print(f"2. Анализ грузов: {public_url.public_url}/cargo_ranking")
        print(f"3. Анализ статусов: {public_url.public_url}/status_analysis")
        print(f"4. Интервалы: {public_url.public_url}/intervals")
        print(f"5. Доступ пассажиров: {public_url.public_url}/healthy_passenger")
        print(f"6. Ранжирование по трафику: {public_url.public_url}/traffic_ranking")
        print(f"7. Топ 5 длительностей: {public_url.public_url}/top_5_longest_maintence")
        print(f"8. Загруженность багажных лент: {public_url.public_url}/baggage_belt_capacity")
        print(f"9. Пересечения в обслуживании: {public_url.public_url}/maintence_intersections")
        print(f"10. Разница в цене: {public_url.public_url}/price_difference")
    except Exception as e:
        print(f"Ошибка ngrok: {str(e)}")
        print("Локальные URL:")
        print("1. http://localhost:8001/max_streak_meal")
        print("2. http://localhost:8001/cargo_ranking")
        print("3. http://localhost:8001/status_analysis")
        print("4. http://localhost:8001/intervals")
        print("5. http://localhost:8001/healthy_passenger")
        print("6. http://localhost:8001/traffic_ranking")
        print("7. http://localhost:8001/top_5_longest_maintence")
        print("8. http://localhost:8001/baggage_belt_capacity")
        print("9. http://localhost:8001/maintence_intersections")
        print("10. http://localhost:8001/price_difference")

INFO:     Started server process [4375]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


Готово! Доступные эндпоинты:
1. Анализ питания: https://cae6-34-32-139-43.ngrok-free.app/max_streak_meal
2. Анализ грузов: https://cae6-34-32-139-43.ngrok-free.app/cargo_ranking
3. Анализ статусов: https://cae6-34-32-139-43.ngrok-free.app/status_analysis
4. Интервалы: https://cae6-34-32-139-43.ngrok-free.app/intervals
5. Доступ пассажиров: https://cae6-34-32-139-43.ngrok-free.app/healthy_passenger
6. Ранжирование по трафику: https://cae6-34-32-139-43.ngrok-free.app/traffic_ranking
7. Топ 5 длительностей: https://cae6-34-32-139-43.ngrok-free.app/top_5_longest_maintence
8. Загруженность багажных лент: https://cae6-34-32-139-43.ngrok-free.app/baggage_belt_capacity
9. Пересечения в обслуживании: https://cae6-34-32-139-43.ngrok-free.app/maintence_intersections
10. Разница в цене: https://cae6-34-32-139-43.ngrok-free.app/price_difference
